https://pintia.cn/problem-sets/1111652100718116864/problems/type/7

In [1]:
import re,time,os
from selenium import webdriver
from pony.orm import *
db = Database()

class Tm(db.Entity):
    lx = Optional(str,column='lx')
    bh = PrimaryKey(str,column='bh')
    url = Optional(str,column='url')
    bt = Optional(str,column='bt')
    fs = Optional(str,column='fs')
    tgs = Optional(str,column='tgs')
    tjs = Optional(str,column='tjs')
    tgl = Optional(str,column='tgl')
    sj = Optional(str,column='sj')
  

In [2]:
def create_database(dbpath):
    if os.path.exists(dbpath):
        os.remove(dbpath)
    f = open(dbpath,"w")
    f.close()
    #db.generate_mapping()
    db.bind(provider='sqlite', filename=dbpath)
    db.generate_mapping(create_tables=True)
    set_sql_debug(True)

In [ ]:
def get_html(url):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)
    html = driver.page_source
    driver.close()
    return html

In [3]:
def extract_list_info(url,category):
    html = get_html(url)
    result = []
    reobj = re.compile(r'<tr><td class="answerIcon_1du7d"></td><td>([\d\D]*?)</td><td><a href="([\d\D]*?)" class="">([\d\D]*?)</a></td><td>([\d\D]*?)</td><td>([\d\D]*?)</td><td>([\d\D]*?)</td><td>([\d\D]*?)</td></tr>')
    for match in reobj.finditer(html):#获取列表页的标号、链接、标题、分数、通过数、提交数、通过率
        biaohao = match.group(1)
        lianjie = "https://pintia.cn/" + match.group(2)
        biaoti = match.group(3)
        fenshu = match.group(4)
        tongguoshu = match.group(5)
        tijiaosshu = match.group(6)
        tongguolv = match.group(7)
        for a,b,c,d,e,f,g in zip(biaohao,lianjie,biaoti,fenshu,tongguoshu,tijiaoshu,tongguolv):#打包在一起
            result.append((category,a,b,c,d,e,f,g))
    return result

In [5]:
def extract_detail_info(results):
    list1 = []
    final_result = []
    for i in results:
        url1=results[2]#从中获取出链接
        html1 = get_html(url1)
        reobj = re.compile(r'<div class="problem_Yd8rq p-3 mb-3 background-grey-1"><div class="ques-view">([\d\D]*?)</div><div class="problemInfo_OZ-Rv">')
        for match in reobj.finditer(html1):
            list1.append(match.group(1))
    for x,y in zip(results,list1):#将详情页的内容与列表页内容拼接
        final_result.append((x,y))
    return final_result

In [6]:
@db_session
def store(final_results):
    for x in final_results:
        s = Ttitle(lx=x[0],bh=x[1],url=x[2],bt=x[3],fs=x[4],tgs=x[5],tjs=x[6],tgl=x[7],sj=x[8])
        db.commit()

In [7]:
def main():
    create_database('D:/code2020/program.sqlite')
    final_results = []
    urls = [("https://pintia.cn/problem-sets/1111652100718116864/problems/type/6","函数题"),("https://pintia.cn/problem-sets/1111652100718116864/problems/type/7","编程题")]
    for url,category in urls:   
        results = extract_list_info(url,category)
        results = extract_detail_info(results)
        final_results.extend(results)
    store(final_results)


In [8]:
# if __name__ == "__main___":
#     main()

main()

OSError: Database file is not found: 'c:/code2020/timu.sqlite'